#Image Classification using Sipeed AIoT.

Sipeed AIoT is an Edge AI MCU which is capable to perform neural network computation at fast speed. In the demo app, Mobilenet V1 model is used and transfer learning is used to train the model to differential between 5 classes of flowers.

This notebook is to combine the steps to create the trained model that can be uploaded to the device.

Acknowledgement:  DmitryM8 and the steps are adapted from [Instructables](https://www.instructables.com/id/Transfer-Learning-With-Sipeed-MaiX-and-Arduino-IDE/) with some modifications.

## Clone the required files from Github

DmitryM8 version of Mobilenet is used.

Additional environtment setup @sistematika


In [ ]:
!pip install segmentation_models
import segmentation_models as sm
sm.set_framework('tf.keras')
sm.framework()

In [ ]:
import tensorflow as tf

In [ ]:
# Clone the libraries to setup the libraries

!git clone https://github.com/AIWintermuteAI/transfer_learning_sipeed.git
!git clone https://github.com/sipeed/Maix_Toolbox.git

### Install the tflite to kmodel conversion software
Note that there is a bug in the get_nncase.sh in the Maix_Toolbox which is unable to extract the file. The steps below is the same as the script but the typo error is fixed.

In [ ]:
%%bash
cd Maix_Toolbox 
mkdir -p ncc
mkdir -p workspace
mkdir -p images
mkdir -p log
cd ncc
wget https://github.com/kendryte/nncase/releases/download/v0.1.0-rc5/ncc-linux-x86_64.tar.xz
tar -Jxf ncc-linux-x86_64.tar.xz
rm ncc-linux-x86_64.tar.xz
echo "download nncase ok!"

## Build the model

In [ ]:
import keras
import numpy as np
from keras import backend as K
from tensorflow.keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense, GlobalAveragePooling2D, Dropout,Flatten


import sys
sys.path.append('/content/transfer_learning_sipeed')
from mobilenet_sipeed.mobilenet import MobileNet

from keras.applications.mobilenet import preprocess_input


### Download the flowers sample dataset

In [ ]:
# Download the flower photos

%cd /content
!curl -LO http://download.tensorflow.org/example_images/flower_photos.tgz
!tar xzf flower_photos.tgz

### Define the parameters

In [ ]:
# the parameters
IMAGE_SIZE = 224
ALPHA=0.75
EPOCHS=20

In [ ]:
def prepare_image(file):
    img_path = ''
    img = image.load_img(img_path + file, target_size=(IMAGE_SIZE, IMAGE_SIZE))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

In [ ]:
# function to define dropout, hidden layers and the number of output
def build_finetune_model(base_model, dropout, fc_layers, num_classes):
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    
    for fc in fc_layers:
        # New FC layer, random init
        x = Dense(fc, activation='relu')(x) 
        x = Dropout(dropout)(x)

    # New softmax layer
    predictions = Dense(num_classes, activation='softmax')(x) 
    
    finetune_model = Model(inputs=base_model.input, outputs=predictions)

    return finetune_model

### Transfer Learning using Mobilenet V1

In [ ]:
# Using MobileNetv1
base_model=MobileNet(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), alpha = ALPHA, 
                     depth_multiplier = 1, dropout = 0.001, include_top = False, 
                     weights = "imagenet", classes = 1000, backend=keras.backend, 
                     layers=keras.layers,models=keras.models,utils=tf.keras.utils)


### Define the last few layers
I used 2 hidden layers and 100 and 50 nodes. More layers or nodes can be added but this will increase the model size and may not fit into Maixpy memory.

In [ ]:
FC_LAYERS = [100, 50]
dropout = 0.5

finetune_model = build_finetune_model(base_model, 
                                      dropout=dropout, 
                                      fc_layers=FC_LAYERS, 
                                      num_classes=5)

In [ ]:
for i,layer in enumerate(finetune_model.layers):
    print(i,layer.name)

In [ ]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('/content/flower_photos',
                                                 target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical', shuffle=True)

### Train the Model

In [ ]:
finetune_model.summary()
finetune_model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
step_size_train=train_generator.n//train_generator.batch_size
history = finetune_model.fit_generator(generator=train_generator,steps_per_epoch=step_size_train,epochs=EPOCHS, shuffle=True)

finetune_model.save('/content/my_model.h5')



In [ ]:
# do a random test to confirm the model is working
preprocessed_image = prepare_image('/content/flower_photos/roses/14494590921_3bb1dc7b88_n.jpg')
predictions_flower = finetune_model.predict(preprocessed_image) 
print(predictions_flower[0][0]*100)    
print(predictions_flower[0][1]*100) 
print(predictions_flower[0][2]*100) 
print(predictions_flower[0][3]*100) 
print(predictions_flower[0][4]*100) 

##Convert the Model
h5 --> tflite --> kmodel

###h5 --> tflite

In [ ]:
#convert keras to tflite format
!tflite_convert  --output_file=/content/model.tflite --keras_model_file=/content/my_model.h5

### tflite --> kmodel

Prepare some test data for ncc util

In [ ]:
%%bash
cd /content

mkdir /content/test_photos/
mkdir /content/test_photos/daisy
mkdir /content/test_photos/dandelion
mkdir /content/test_photos/roses
mkdir /content/test_photos/sunflowers
mkdir /content/test_photos/tulips


In [ ]:
import os
import shutil


def recursive_copy(src, dest):
    """
    Copy each file from src dir to dest dir, including sub-directories.
    """
    
    #copy only 10 files
    count=0
    for item in os.listdir(src):
        file_path = os.path.join(src, item)

        # if item is a file, copy it
        if os.path.isfile(file_path):
            shutil.copy(file_path, dest)

        # else if item is a folder, recurse 
        elif os.path.isdir(file_path):
            new_dest = os.path.join(dest, item)
            os.mkdir(new_dest)
            recursive_copy(file_path, new_dest)
        if count == 10:
          break
        count+=1

recursive_copy("/content/flower_photos/daisy", "/content/test_photos/daisy")
recursive_copy("/content/flower_photos/dandelion", "/content/test_photos/dandelion")
recursive_copy("/content/flower_photos/roses", "/content/test_photos/roses")
recursive_copy("/content/flower_photos/sunflowers", "/content/test_photos/sunflowers")
recursive_copy("/content/flower_photos/tulips", "/content/test_photos/tulips")



In [ ]:
#convert tflite to kmodel format
# this will take some time...
%cd /content/Maix_Toolbox
!./ncc/ncc -i tflite -o k210model --dataset /content/test_photos /content/model.tflite /content/model.kmodel

### Download the generated file

In [ ]:
from google.colab import files

files.download('/content/model.kmodel')

After this step,

download the kmodel to the  

1.   Download the kmodel file in /content/model.kmodel
2.   Create a label file. For the flower dataset the content of the label file is

daisy

dandelion

roses

sunflowers

tulips



This is the micropython code to be executed on the Sipeed devices. 
(Do not run this in Colab)

In [ ]:
import sensor, image, lcd, time
import KPU as kpu
lcd.init()
sensor.reset()
sensor.set_pixformat(sensor.RGB565)
sensor.set_framesize(sensor.QVGA)
sensor.set_windowing((224, 224))
sensor.set_vflip(0)
sensor.set_hmirror(0)
sensor.run(1)
lcd.clear()
lcd.draw_string(100,96,"MobileNet Demo")
lcd.draw_string(100,112,"Loading labels...")
f=open('flowerlabel.txt','r')
labels=f.readlines()
f.close()
task = kpu.load("/sd/model.kmodel") 
clock = time.clock()
while(True):
    img = sensor.snapshot()
    clock.tick()
    fmap = kpu.forward(task, img)
    fps=clock.fps()
    plist=fmap[:]
    pmax=max(plist)	
    max_index=plist.index(pmax)	
    a = lcd.display(img, oft=(50,0))
    lcd.draw_string(0, 224, "%.2f:%s                            "%(pmax, labels[max_index].strip()))
    print(pmax)
a = kpu.deinit(task)

This is the [Youtube Video](https://www.youtube.com/watch?v=0Pc0LwmRWRk) for the realtime inferencing using Sipeed Maixpy Go